In [1]:
from collections import namedtuple
import csv
import psycopg2
import sys
from razi.rdkit_postgresql.types import Mol

from rdkit import Chem, rdBase
from rdkit.Chem import Draw

from sqlalchemy import create_engine, Column, Index, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


print(sys.version_info)
print(f'RDKit version: {rdBase.rdkitVersion}')

sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)
RDKit version: 2017.03.1


#### 1. データベースに接続する

`create_engine`関数を用いてPostgreSQLに接続する。

In [2]:
engine = create_engine('postgresql://postgres@db:5432/postgres')

### 1. テーブルの定義

[SQLAlchemy](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html)でテーブルの定義をする方法と同じように`Razi`でも定義することができる。  
その際カラムのデータ型として化学構造データを保存する`Mol`が追加されている。

最初にテーブルを定義するクラスが継承する`Base`クラスを呼び出す。

In [3]:
Base = declarative_base(bind=engine)

例えば下の例では`id`, `name`, `structure`という名前のカラムを持つテーブル`compounds`を作成している。  
`__tablename__`アトリビュート(JAVAでいうフィールド)にテーブルの名前`compounds`を保存し、  
各カラムはアトリビュートに初期値として`Column`クラスをあたえることでカラムとして定義することができる。  
さらに`Column`クラスのコンストラクタの引数に`Integer`, `String`, `Mol`を与えることで  
それぞれ整数値、文字列、化学構造を保存するカラムであると定義している。  
`id`にはさらに引数として`primary_key=True`を与えることでPrimary key(主キー)であると定義している。

In [4]:
class Compound(Base):
    __tablename__ = 'compounds'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    structure = Column(Mol)
    
    __table_args__ = (
        Index('compounds_structure', 'structure',
                    postgresql_using='gist'),
        )
    
    def __init__(self, name, structure):
        self.name = name
        self.structure = structure
        
    def __repr__(self):
        return f'({self.name}) < {self.structure} >'

### 2. テーブルの構築

1で定義したテーブルを実際に構築するには以下の関数を実行する。

In [5]:
Base.metadata.create_all()

### 3. データの登録

CHEMBLのデータである`tutorial_compounds.txt`は以下のような形式で保存されている。

In [15]:
!head -n3 tutorial_compounds.txt

chembl_id	canonical_smiles	standard_inchi	standard_inchi_key
CHEMBL153534	Cc1cc(cn1C)c2csc(N=C(N)N)n2	InChI=1S/C10H13N5S/c1-6-3-7(4-15(6)2)8-5	<snip>
CHEMBL25	CC(=O)Oc1ccccc1C(=O)O	InChI=1S/C9H8O4/c1-6(10)13-8-5-3-2-4-7(8)9(11)12/h2-5H,1H3,(H,11,12)	BSYNRYMUTXBXSQ-UHFFFAOYSA-N


このファイルからvalidなsmilesとそれに対応する`CHEMBL ID`そして新たな`ID`を割り当てる関数`read_chembldb`を以下のように作成した。

In [7]:
Record = namedtuple('Record', 'chembl_id, smiles, inchi, inchi_key')


def read_chembldb(filepath, limit=0):
    with open(filepath, 'rt') as inputfile:
            reader = csv.reader(inputfile, delimiter='\t', skipinitialspace=True)
            #headerを飛ばす
            next(reader)

            for count, record in enumerate(map(Record._make, reader), start=1):
                smiles = record.smiles

                #特定の三重結合をRDKitで読めるように変換する。
                smiles = smiles.replace('=N#N','=[N+]=[N-]')
                smiles = smiles.replace('N#N=','[N-]=[N+]=')            

                #invalidなsmilesは読み込まない
                if not Chem.MolFromSmiles(smiles):
                    continue

                yield count, record.chembl_id, smiles
                if count == limit:
                    break

期待する通りに関数が動いているか確かめる。関数`read_chembldb`は引数`limit`の数だけデータを取り出す(デフォルトは全てのデータを取り出す)。  
今回は`limit=3`で行ってみる。

In [16]:
for count, chembl_id, smiles in read_chembldb('tutorial_compounds.txt', limit=3):
    print(count, chembl_id, smiles)

1 CHEMBL153534 Cc1cc(cn1C)c2csc(N=C(N)N)n2
2 CHEMBL25 CC(=O)Oc1ccccc1C(=O)O
4 CHEMBL674 CCC(CC)O[C@@H]1C=C(C[C@H](N)[C@H]1NC(=O)C)C(=O)O
5 CHEMBL3638313 CC(=O)C[C@H](C1=C(O)c2ccc(OC3O[C@@H]([C@@H](O)[C@H](O)[C@H]3O)C(=O)O)cc2OC1=O)c4ccccc4


期待する通りに関数が動いているようだ。

データベースに接続し、検索や登録するには`sessionmaker`を用いる。

In [9]:
Session = sessionmaker(bind=engine)

データを実際に登録してみる。定義したテーブル`Compound` classから一般的なオブジェクト指向型プログラミングにおけるオブジェクトを作成し、  
sessionオブジェクトの`add`メソッドを用いて登録すれば良い。

In [17]:
session = Session()
for count, chembl_id, smiles in read_chembldb('tutorial_compounds.txt', 10):
    compound = Compound(chembl_id, smiles)
    session.add(compound)
session.commit()
session.close()

また別の登録方法としてRDkitのMolオブジェクトを引数に与える方法がある。

In [13]:
session = Session()
smiles = 'c1ccccc1Cl'
mol =Chem.MolFromSmiles(smiles)
compound = Compound('111111', mol)
session.add(compound)
session.commit()
session.close()

(111111) < <rdkit.Chem.rdchem.Mol object at 0x7f4f3cdf4a80> >
